In [1]:
import glob
import numpy as np
from music21 import converter,instrument,note,chord,stream
import pickle
import keras
from keras.utils import to_categorical

Using TensorFlow backend.
/home/avinash/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/avinash/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/avinash/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/avinash/.local/lib/python3.6/site-packages/tensorflow/py

# Get Notes from Music Files

In [2]:
midi = converter.parse("midi_songs/ahead_on_our_way_piano.mid")

In [3]:
#midi.show("midi")

In [4]:
#midi.show("text")

In [5]:
notes_to_parse = midi.flat.notes

In [6]:
len(notes_to_parse)


720

In [7]:
for i in notes_to_parse:
    print(i,i.offset)

<music21.chord.Chord C4 E4> 0.0
<music21.chord.Chord C4 E4> 0.5
<music21.chord.Chord C4 F4> 1.0
<music21.chord.Chord C4 F4> 1.5
<music21.chord.Chord C4 E4> 2.0
<music21.chord.Chord C4 E4> 2.5
<music21.chord.Chord C4 D4> 3.0
<music21.note.Note C> 3.5
<music21.chord.Chord C4 D4> 3.5
<music21.note.Note F> 3.75
<music21.note.Note G> 4.0
<music21.chord.Chord C4 E4> 4.0
<music21.chord.Chord C4 E4> 4.5
<music21.chord.Chord C4 F4> 5.0
<music21.chord.Chord C4 F4> 5.5
<music21.chord.Chord C4 E4> 6.0
<music21.chord.Chord C4 E4> 6.5
<music21.chord.Chord C4 D4> 7.0
<music21.chord.Chord C4 D4> 7.5
<music21.chord.Chord C4 E4> 8.0
<music21.chord.Chord C4 E4> 8.5
<music21.chord.Chord C4 F4> 9.0
<music21.chord.Chord C4 F4> 9.5
<music21.chord.Chord C4 E4> 10.0
<music21.chord.Chord C4 E4> 10.5
<music21.chord.Chord C4 D4> 11.0
<music21.note.Note F> 11.5
<music21.chord.Chord C4 D4> 11.5
<music21.note.Note B-> 11.75
<music21.note.Note C> 12.0
<music21.chord.Chord C4 E4> 12.0
<music21.chord.Chord C4 E4> 12.5


In [8]:
notes_demo = []

for element in notes_to_parse:
    # if the element is a note, then store it's pitch
    if isinstance(element,note.Note):
        notes_demo.append(str(element.pitch))
        
    if isinstance(element,chord.Chord):
        notes_demo.append("+".join(str(n) for n in element.normalOrder))
         

In [9]:
notes = []
for file in glob.glob("midi_songs/*.mid"):
    midi = converter.parse(file)
    
    print("parsing %s "%file)
    notes_to_parse = None
    notes_to_parse = midi.flat.notes
    
    for element in notes_to_parse:
        # if the element is a note, then store it's pitch
        if isinstance(element,note.Note):
            notes.append(str(element.pitch))
        # if element is chord    
        if isinstance(element,chord.Chord):
            notes.append("+".join(str(n) for n in element.normalOrder))
         

parsing midi_songs/Ff4-BattleLust.mid 
parsing midi_songs/FF6epitaph_piano.mid 
parsing midi_songs/relmstheme-piano.mid 
parsing midi_songs/waltz_de_choco.mid 
parsing midi_songs/Fiend_Battle_(Piano).mid 
parsing midi_songs/Kingdom_Hearts_Traverse_Town.mid 
parsing midi_songs/Finalfantasy6fanfarecomplete.mid 
parsing midi_songs/Eternal_Harvest.mid 
parsing midi_songs/FF3_Battle_(Piano).mid 
parsing midi_songs/rufus.mid 
parsing midi_songs/FF8_Shuffle_or_boogie_pc.mid 
parsing midi_songs/FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid 
parsing midi_songs/8.mid 
parsing midi_songs/lurk_in_dark.mid 
parsing midi_songs/ViviinAlexandria.mid 
parsing midi_songs/Oppressed.mid 
parsing midi_songs/thenightmarebegins.mid 
parsing midi_songs/z_aeristhemepiano.mid 
parsing midi_songs/ff4-airship.mid 
parsing midi_songs/AT.mid 
parsing midi_songs/BlueStone_LastDungeon.mid 
parsing midi_songs/braska.mid 
parsing midi_songs/tpirtsd-piano.mid 
parsing midi_songs/Fyw_piano.mid 
parsing midi_songs/

In [10]:
with open("notes","wb") as f:
    pickle.dump(notes, f)

In [11]:
with open("notes","rb") as f:
    notes = pickle.load(f)

In [12]:
len(notes)

60498

In [13]:
n_vocab= len(set(notes))

# Prepare Sequential Data for LSTM

In [14]:
sequence_length = 100

In [15]:
pitchnames = sorted(list(set(notes)))

In [16]:
note_to_int = dict((note,num) for num, note in enumerate(pitchnames)) 

In [17]:
network_input = []
network_output = []

In [18]:
for i in range(len(notes) - sequence_length):
    sequence_in = notes[i:i+sequence_length]
    sequence_out = notes[i+sequence_length]
    
    network_input.append([note_to_int[ch] for ch in sequence_in])
    network_output.append(note_to_int[sequence_out])

In [19]:
n_patterns = len(network_input)

In [35]:
normalized_network_input = np.reshape(network_input,(n_patterns, 100,1))
normalized_network_input.shape

(60398, 100, 1)

In [36]:
normalized_network_input = normalized_network_input/float(n_vocab)

In [37]:
network_output = to_categorical(network_output)

In [38]:
normalized_network_input.shape

(60398, 100, 1)

In [39]:
network_output.shape

(60398, 359, 2)

# Create Model

In [25]:
from keras.models import Sequential, load_model
from keras.layers import Dense,Dropout,LSTM,Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [41]:
x = normalized_network_input.shape[1]
y = normalized_network_input.shape[2]
X = normalized_network_input

In [32]:
model = Sequential()
model.add(LSTM(512,input_shape=(x,y),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512,return_sequences=True))
model.add(Dropout(0,3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='rmsprop')

In [33]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)              

In [43]:
model = load_model("weights-improvement-60-1.1406-bigger.hdf5")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



# Predictions

In [44]:
start = np.random.randint(len(network_input))

In [45]:
int_to_note = dict((num,note) for num, note in enumerate(pitchnames)) 

In [46]:
pattern = network_input[start]
prediction_out = []


for note_index in range(100):
    prediction_input = np.reshape(pattern,(1,len(pattern),1))
    prediction_input = prediction_input/float(n_vocab)
    
    pred = model.predict(prediction_input)
    
    index = np.argmax(pred)
    result = int_to_note[index]
    prediction_out.append(result)
    
    
    # remove first value, and append the recent prediction
    # moving forward set y step with time/.....
    pattern.append(index)
    pattern = pattern[1:]

In [47]:
print(prediction_out)

['2+5', 'C5', 'B4', '2+5', '2+5', 'C5', '0+5', '2+5', '0+5', '0+5', '0+5', '0+5', '0+5', '0+5', '0+4', '2+5', 'G5', 'B4', '2+5', 'F5', '2+5', '2+5', '0+5', '2+5', '0+5', '0+5', '0+5', '0+5', '0+5', '0+5', 'G5', '2+5', '0+5', 'F5', 'C5', 'G5', 'E5', '0+5', 'G5', '2+5', '0+5', '0+5', '2+5', 'C5', '0+5', '0+5', '0+5', '2+5', '2+5', '0+4', '0+5', 'F5', '0+4', '0+5', '2+5', '0+5', '0+5', '2+5', '2+5', '0+4', 'F5', 'F5', 'B4', '0+5', '2+5', '2+5', '2+5', 'C5', '0+5', '0+5', '0+5', '2+5', '0+5', '0+5', '0+5', '0+5', '0+5', '0+5', '0+5', '0+5', '0+5', 'B4', '0+5', '0+5', '0+5', '0+5', 'C5', '5+10', 'C5', 'B4', '2+5', '0+5', 'B4', '0+5', '2+5', 'D2', '0+5', '0+5', 'F5', '0+5']


# Create midi file

In [48]:
offset = 0
output_notes = []

for pattern in prediction_out:
    #check if it is a chord
    if('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split("+")
        notes_obj = []
        
        for cur_note in notes_in_chord:
            new_note = note.Note(int(cur_note))
            new_note.storedInstrument = instrument.Piano()
            notes_obj.append(new_note)
        new_chord = chord.Chord(notes_obj)
        new_chord.affset = offset
        output_notes.append(new_chord)
    #check if it is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        offset += 0.5

In [49]:
output_notes

[<music21.chord.Chord D F>,
 <music21.note.Note C>,
 <music21.note.Note B>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.note.Note C>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C E>,
 <music21.chord.Chord D F>,
 <music21.note.Note G>,
 <music21.note.Note B>,
 <music21.chord.Chord D F>,
 <music21.note.Note F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C F>,
 <music21.note.Note G>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord C F>,
 <music21.note.Note F>,
 <music21.note.Note C>,
 <music21.note.Note G>,
 <music21.note.Note E>,
 <music2

In [50]:
midi_stream = stream.Stream(output_notes)

In [51]:
midi_stream

<music21.stream.Stream 0x7f09ea7ffbe0>

In [52]:
midi_stream.show("midi")

In [53]:
midi_stream.write("midi","output.mid")

'output.mid'